## Tempus Bioinformatics Coding Challenge:

- See ./challenge_docs/ for challenge instructions and .vcf file.
- functions used within this notebook can be seen in ./functions/

Note: Instructions state, "If multple [variant] possibilities, annotate with the most deleterious possibility". As the provided VCF was collapsed on variant position, different variants were aggregated into the same vcf line.  As arbitrary determination of deleterious variants can be perfidious, 2 output tables were genereated:

| File Name       | Description           |
| ------------- |:-------------:|
| ./output/annotated_Tempus_BIIN_Challenge_SR_3-19-2019.csv    | All variants broken into separate descrete values and annotated. | 
| ./output/annotated_Tempus_BIIN_Challenge_most_deleterious_SR_3-19-2019.csv      | Annotation of most deleterious variant at a given position |   


In [1]:
##IMPORTS 
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np

#need to import local path to import module
module_path = os.path.abspath(os.path.join('./functions'))
if module_path not in sys.path:
    sys.path.append(module_path)
     
#import function for reading VCF in to DF
from vcf_explode import VCFexplode, readVCF
from explode_DF import explode
from parse_variant_functions import parse_exac_json, df_exac_variant_GET

In [2]:
#read VCF into DF with VCF explode imp|orted above:
vcfDF = VCFexplode("./challege_docs/Challenge_data.vcf", explode_info=True, explode_fmt=True)
# Now we have a pandas DF of the VCF, we want to make a subtable- lets make a function for that
vcfDF_small =  vcfDF[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'TYPE', 'DP', 'AF', 'AO']] #keep standard vcf columns, as well as additional ones challenge is looking for.
vcfDF_small_working = vcfDF_small.copy()
vcfDF_small_working.head() # peek to see if it is what we expect

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,TYPE,DP,AF,AO
0,1,931393,.,G,T,2.17938e-13,.,snp,4124,0,95
1,1,935222,.,C,A,16866.7,.,snp,1134,0.666667,652
2,1,1277533,.,T,C,28168.6,.,snp,786,1,786
3,1,1284490,.,G,A,6300.56,.,snp,228,1,228
4,1,1571850,.,G,A,0,.,snp,4055,0,94


In [3]:
#turn string values with commas into lists- so we can break them up. Easier to handle descrete data values.
vcfDF_small_working.loc[:,"ALT"] = vcfDF_small_working["ALT"].str.split(",")
vcfDF_small_working.loc[:,"AF"] = vcfDF_small_working["AF"].str.split(",")
vcfDF_small_working.loc[:,"AO"] = vcfDF_small_working["AO"].str.split(",")
vcfDF_small_working.loc[:,"TYPE"] = vcfDF_small_working["TYPE"].str.split(",")
vcfDF_small_working.head() # peek to see if it is what we expect

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,TYPE,DP,AF,AO
0,1,931393,.,G,[T],2.17938e-13,.,[snp],4124,[0],[95]
1,1,935222,.,C,[A],16866.7,.,[snp],1134,[0.666667],[652]
2,1,1277533,.,T,[C],28168.6,.,[snp],786,[1],[786]
3,1,1284490,.,G,[A],6300.56,.,[snp],228,[1],[228]
4,1,1571850,.,G,[A],0,.,[snp],4055,[0],[94]


In [4]:
#get separate line for each detected variant:
vcfDF_expl = explode(vcfDF_small_working, lst_cols = ['ALT', 'AF', 'AO', "TYPE"], fill_value='', preserve_index=False )
#lets get ourselves an alt percent
vcfDF_expl["ref_read_fraction"] =  1 - (vcfDF_expl["AO"].astype(int) / vcfDF_expl["DP"].astype(int)) #ref_read_fraction: ratio of reads supporting Reference
vcfDF_expl["alt_read_fraction"] =  vcfDF_expl["AO"].astype(int) / vcfDF_expl["DP"].astype(int) #ref_read_fraction: ratio of reads supporting Alt
#reorder columns to more normalized vcf behavior.
vcfDF_expl = vcfDF_expl[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'TYPE', 'DP', 'AF', 'AO', "ref_read_fraction", "alt_read_fraction"]]
vcfDF_expl.head() # peek to see if it is what we expect

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,TYPE,DP,AF,AO,ref_read_fraction,alt_read_fraction
0,1,931393,.,G,T,2.17938e-13,.,snp,4124,0,95,0.976964,0.023036
1,1,935222,.,C,A,16866.7,.,snp,1134,0.666667,652,0.425044,0.574956
2,1,1277533,.,T,C,28168.6,.,snp,786,1,786,0.000000,1.000000
3,1,1284490,.,G,A,6300.56,.,snp,228,1,228,0.000000,1.000000
4,1,1571850,.,G,A,0,.,snp,4055,0,94,0.976819,0.023181


In [5]:
#add Json info, then transform. 
vcfDF_expl["json"] = vcfDF_expl.apply(lambda row: parse_exac_json(df_exac_variant_GET(row["CHROM"],row["POS"],row["REF"],row["ALT"])), axis=1)
vcfDF_expl = vcfDF_expl.join(pd.DataFrame(vcfDF_expl["json"].to_dict()).T).fillna("") #transform dictionary of useful items to dataframe columns
vcfDF_expl.drop("json", inplace=True, axis=1) # drop Json column as it has been pulled into the larger df. 
vcfDF_expl.head() #peek to see if it is what we expect

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,TYPE,DP,AF,...,canonical_transcript,exac_AF,gene_sym,hgvs_cdna_mut,hgvs_prot_mut,polyphen,sift,strand,var_exon_number,variant_type
0,1,931393,.,G,T,2.17938e-13,.,snp,4124,0,...,,,,,,,,,,
1,1,935222,.,C,A,16866.7,.,snp,1134,0.666667,...,ENSG00000188290,0.661111,HES4,ENST00000428771.2:c.132G>T,ENSP00000393198.2:p.Arg44Ser,benign(0),tolerated(0.95),-1,1/3,missense_variant
2,1,1277533,.,T,C,28168.6,.,snp,786,1,...,ENSG00000107404,0.997571,DVL1,ENST00000378891.5:c.366A>G,ENST00000378891.5:c.366A>G(p.%3D),,,-1,4/15,synonymous_variant
3,1,1284490,.,G,A,6300.56,.,snp,228,1,...,ENSG00000107404,0.897299,DVL1,ENST00000378891.5:c.-45C>T,,,,-1,1/15,5_prime_UTR_variant
4,1,1571850,.,G,A,0,.,snp,4055,0,...,ENSG00000248333,1.11376e-05,CDK11B,ENST00000407249.3:c.1927-7C>T,,,,-1,,splice_region_variant&intron_variant


In [6]:
#now lets output the table. 
vcfDF_expl.to_csv(Path("./outputs/annotated_Tempus_BIIN_Challenge_SR_3-19-2019.csv"))
#if excel is preferred can do -> to excel.  

### Now we have a DF of all vars output, lets trim out repeat variants to have a single most deleterious variant per site:

In [7]:
vcfDF_deleterious = vcfDF_expl.copy() #create a copy to work from
#set TYPE category as categorical so we can order them
vcfDF_deleterious['TYPE'] = pd.Categorical(vcfDF_deleterious['TYPE'], ["ins", "del", "complex", "mnp", "snp"], ordered=True)
#sort by TYPE to drop other instances:
vcfDF_deleterious = vcfDF_deleterious.sort_values("TYPE")
#perform Drop of extra columns
vcfDF_deleterious.drop_duplicates(["CHROM", "POS"], keep="first", inplace=True )
#resort by chrom position, and reset the index.
vcfDF_deleterious.sort_values(["CHROM", "POS"], inplace=True)
vcfDF_deleterious.reset_index(inplace=True) # want to reset index
vcfDF_deleterious.head() #take a look

,index,CHROM,POS,ID,REF,ALT,QUAL,FILTER,TYPE,DP,...,canonical_transcript,exac_AF,gene_sym,hgvs_cdna_mut,hgvs_prot_mut,polyphen,sift,strand,var_exon_number,variant_type
0,0,1,931393,.,G,T,2.17938e-13,.,snp,4124,...,,,,,,,,,,
1,1,1,935222,.,C,A,16866.7,.,snp,1134,...,ENSG00000188290,0.661111,HES4,ENST00000428771.2:c.132G>T,ENSP00000393198.2:p.Arg44Ser,benign(0),tolerated(0.95),-1,1/3,missense_variant
2,2,1,1277533,.,T,C,28168.6,.,snp,786,...,ENSG00000107404,0.997571,DVL1,ENST00000378891.5:c.366A>G,ENST00000378891.5:c.366A>G(p.%3D),,,-1,4/15,synonymous_variant
3,3,1,1284490,.,G,A,6300.56,.,snp,228,...,ENSG00000107404,0.897299,DVL1,ENST00000378891.5:c.-45C>T,,,,-1,1/15,5_prime_UTR_variant
4,4,1,1571850,.,G,A,0,.,snp,4055,...,ENSG00000248333,1.11376e-05,CDK11B,ENST00000407249.3:c.1927-7C>T,,,,-1,,splice_region_variant&intron_variant


In [10]:
#output file to outputs folder:
vcfDF_deleterious.to_csv(Path("./outputs/annotated_Tempus_BIIN_Challenge_most_deleterious_SR_3-19-2019.csv"))